In [13]:
import tensorflow as tf
import sys
sys.path.append("../")
from federated.models.mitbih_model import create_cnn_model, create_dense_model
from federated.utils.mitbih_data_preprocessing import load_data
from sklearn.metrics import confusion_matrix
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Loading model and test dataset

In [7]:
model = tf.keras.models.load_model("../history/logdir/test_v2")
X_test, y_test = load_data(data_analysis=True)

X_test = X_test.reshape(len(X_test), X_test.shape[1],1)
X_test.shape

(21892, 186, 1)

## Evaluating Model

In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy: {scores[1]*100}%, Loss: {scores[0]}")

Accuracy: 65.59473872184753%, Loss: 1.8306316137313843


In [9]:
y_pred = model.predict(X_test)

In [11]:
y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

## Confusion Matrix

In [12]:
cnf_matrix = confusion_matrix(y_test, y_pred)

labels = ["N", "S", "V", " F", "U"]

plt.figure(figsize=(15,15))
cnf_matrix =  cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
plt.imshow(cnf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Normalized Confusion Matrix")
plt.colorbar()
ticks = np.arange(len(labels))


    